## Passing arguments to threads

In [ ]:
def task(num):
    print(f"I'm the task {num}")

for i in range(10):
    t = threading.Thread(target=task, args=(i,))
    t.start()

## [Thread synchronization](https://pymotw.com/3/threading/index.html#signaling-between-threads)

In [3]:
import logging
import threading

def wait_for_an_event(event):
    print("I'm doing something")
    logging.debug("waitting for an event from other thread ...")
    e = event.wait()
    logging.debug(f"received event {e}")

def wait_for_an_event_with_timeout(event, timeout):
    while not event.is_set():
        print("I'm also doing something")
        logging.debug('waitting for an event from other tread, but with a timeout ...')
        e = event.wait(timeout)
        logging.debug(f"received event {e}")

logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

e = threading.Event()
threading.Thread(target=wait_for_an_event, args=(e,),).start()
threading.Thread(target=wait_for_an_event_with_timeout, args=(e, 2),).start()

logging.debug('waiting before calling Event.set()')
e.set()
logging.debug('event is set')

(Thread-6  ) waitting for an event from other thread ...
(Thread-7  ) waitting for an event from other tread, but with a timeout ...
(MainThread) waiting before calling Event.set()
(MainThread) event is set


I'm doing something
I'm also doing something


(Thread-6  ) received event True
(Thread-7  ) received event True


## Using [critical sections](https://en.wikipedia.org/wiki/Critical_section)

All Python’s built-in data structures (such as lists, dictionaries, etc.) are thread-safe. However, other user's data structures implemented in Python, or simpler types like integers and floats, cannot be accesed concurrently.

In [ ]:
import logging
import random
import threading
import time

class Counter:
    
    def __init__(self, start=0):
        self.lock = threading.Lock()
        self.value = start

    def increment(self):
        logging.debug('Waiting for lock')
        self.lock.acquire()
        try:
            logging.debug('Acquired lock')
            self.value = self.value + 1
        finally:
            self.lock.release()

def worker(c):
    for i in range(2):
        pause = random.random()
        logging.debug('Sleeping %0.02f', pause)
        time.sleep(pause)
        c.increment()
    logging.debug('Done')


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

counter = Counter()
for i in range(2):
    t = threading.Thread(target=worker, args=(counter,))
    t.start()

logging.debug('Waiting for worker threads')
main_thread = threading.main_thread()
for t in threading.enumerate():
    if t is not main_thread:
        t.join()
logging.debug('Counter: %d', counter.value)

## 2. Processes
[Processes](https://en.wikipedia.org/wiki/Process_%28computing%29) allow the parallel execution of code in [multiprocessing systems](https://en.wikipedia.org/wiki/Multiprocessing). This can be used to solve the previously mentioned limitation of the GIL.

In [ ]:
# This code does not work!

import multiprocessing
import time
import sys

a = ''

def task_1():
    global a
    for i in range(10):
        print('o', end='')
        sys.stdout.flush()
        a += 'o'
        print(a)
        time.sleep(1)
        
def task_2():
    global a
    for i in range(20):
        print('O', end='')
        sys.stdout.flush()
        a += 'O'
        print(a)
        time.sleep(0.6)
        
process_1 = multiprocessing.Process(target=task_1)
process_2 = multiprocessing.Process(target=task_2)

process_1.start()
process_2.start()
print("(Both processes have started)")

process_1.join()
process_2.join()
print("\nBoth processes have finished")

print(a)

### But ... why `a` has not been modified? Why the processed do not share `a`?

By definition, processes must [__fork__](https://en.wikipedia.org/wiki/Fork_(system_call) (make a copy of itself, that is, the code and the used memory) before start running. In the previous example, the Python interpreter forks twice and the two childs are run in parallel while the parent process waits for their completition. Neither, the child processes nor the parent process share their global state (where `a` is defined).

### 2.1. How to share data between processes?

There are several options. One of them is to use a [shared memory `Manager()`](https://docs.python.org/3/library/multiprocessing.html#sharing-state-between-processes):

In [ ]:
import multiprocessing
import time
import sys
import ctypes

def task_1(a):
    for i in range(10):
        print('o', end='')
        sys.stdout.flush()
        a.value += 'o'
        time.sleep(1)
        
def task_2(a):
    for i in range(20):
        print('O', end='')
        sys.stdout.flush()
        a.value += 'O'
        time.sleep(0.6)

manager = multiprocessing.Manager()
# See https://docs.python.org/3/library/ctypes.html#module-ctypes
a = manager.Value(ctypes.c_char_p, "")    

process_1 = multiprocessing.Process(target=task_1, args=(a,))
process_2 = multiprocessing.Process(target=task_2, args=(a,))

process_1.start()
process_2.start()
print("(Both processes have started)")

process_1.join()
process_2.join()
print("\nBoth processes have finished")

print(a.value)

## 3. Using critical sections
A [critical section](https://en.wikipedia.org/wiki/Linearizability) is a region of code that should not run in parallel. For example, the increment of a variable is not considered an [atomic operation](https://en.wikipedia.org/wiki/Linearizability), so, it should be performed using [mutual exclusion](https://en.wikipedia.org/wiki/Mutual_exclusion).

Example of why critical sections must be executed using mutual exclusion:

In [ ]:
# Two threads that have a critical section executed in parallel without mutual exclusion.
# This code does not work!

import threading
import time
import sys

counter = 10

def task_1():
    global counter
    for i in range(1000000):
        counter += 1
        
def task_2():
    global counter
    for i in range(1000001):
        counter -= 1
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

thread_1.start()
thread_2.start()
print("(Both threads started)")

thread_1.join()
thread_2.join()
print("\nBoth threads finished")

print('counter =', counter)

The same example, using mutual exclusion (using a [lock](https://docs.python.org/3/library/threading.html#lock-objects)):

In [ ]:
# Two threads that have a critical section executed sequentially.

import threading
import time
import sys

lock = threading.Lock()
counter = 10

def task_1():
    global counter
    for i in range(1000000):
        with lock:
            counter += 1
        
def task_2():
    global counter
    for i in range(1000001):
        with lock:
            counter -= 1
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

thread_1.start()
thread_2.start()
print("(Both threads started)")

thread_1.join()
thread_2.join()
print("\nBoth threads finished")

print('counter =', counter)

Same concept, using processes:

In [ ]:
# Two processes that have a critical section executed sequentially

import multiprocessing
import time
import sys
import ctypes

def task_1(lock, counter):
    for i in range(10000):
        lock.acquire()
        try:
            counter.value += 1
        finally:
            lock.release()
        
def task_2(lock, counter):
    for i in range(10001):
        lock.acquire()
        try:
            counter.value -= 1
        finally:
            lock.release()

lock = multiprocessing.Lock()

manager = multiprocessing.Manager()
counter = manager.Value(ctypes.c_int, 10)

process_1 = multiprocessing.Process(target=task_1, args=(lock, counter))
process_2 = multiprocessing.Process(target=task_2, args=(lock, counter))

process_1.start()
process_2.start()
print("(Both tasks started)")

process_1.join()
process_2.join()
print("\nBoth tasks finished")

print('counter =', counter.value)

## 4. Synchronizing parallel tasks

### 4.1. Using a queue

With threads:

In [ ]:
import threading
import time
import random
import queue

queue_length = 10
q = queue.Queue(queue_length)

class Producer(threading.Thread):
    def __init__(self, iters=10):
        super(Producer,self).__init__()
        self.iters = iters

    def run(self):
        i = 0
        while i < self.iters:
            if not q.full():
                item = random.randint(1,10)
                q.put(item)
                print('Produced {} (queue length = {})'.format(item,q.qsize()))
                i += 1
                time.sleep(random.random())
        return

class Consumer(threading.Thread):
    def __init__(self, iters=10):
        super(Consumer,self).__init__()
        self.iters = iters

    def run(self):
        i = 0
        while i < self.iters:
            if not q.empty():
                item = q.get()
                print('Consumed {} (queue length = {})'.format(item,q.qsize()))
                i += 1
                time.sleep(random.random())
        return

p = Producer()
p.start()import logging
import random
import threading
import time


class Counter:

    def __init__(self, start=0):
        self.lock = threading.Lock()
        self.value = start

    def increment(self):
        logging.debug('Waiting for lock')
        self.lock.acquire()
        try:
            logging.debug('Acquired lock')
            self.value = self.value + 1
        finally:
            self.lock.release()


def worker(c):
    for i in range(2):
        pause = random.random()
        logging.debug('Sleeping %0.02f', pause)
        time.sleep(pause)
        c.increment()
    logging.debug('Done')


logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

counter = Counter()
for i in range(2):
    t = threading.Thread(target=worker, args=(counter,))
    t.start()

logging.debug('Waiting for worker threads')
main_thread = threading.main_thread()
for t in threading.enumerate():
    if t is not main_thread:
        t.join()
logging.debug('Counter: %d', counter.value)
c = Consumer()
c.start()

p.join()
c.join()

print('done')

With processes:

In [ ]:
import multiprocessing
import time
import random

class Producer():
    def __init__(self, q, iters=10):
        super(Producer,self).__init__()
        self.iters = iters
        self.q = q

    def run(self):
        i = 0
        while i < self.iters:
            if not self.q.full():
                item = random.randint(1,10)
                self.q.put(item)
                print('Produced {} (queue length = {})'.format(item, self.q.qsize()))
                i += 1
                time.sleep(random.random())
        return
    
    def start(self):
        process = multiprocessing.Process(target=self.run)
        process.start()
        return process

class Consumer():
    def __init__(self, q, iters=10):
        super(Consumer,self).__init__()
        self.iters = iters
        self.q = q

    def run(self):
        i = 0
        while i < self.iters:
            if not self.q.empty():
                item = self.q.get()
                print('Consumed {} (queue length = {})'.format(item, self.q.qsize()))
                i += 1
                time.sleep(random.random())
        return

    def start(self):
        process = multiprocessing.Process(target=self.run)
        process.start()
        return process

queue_length = 10
q = multiprocessing.Queue(queue_length)

p = Producer(q)
task_p = p.start()
c = Consumer(q)
task_c = c.start()

task_p.join()
task_c.join()

print('done')